# Network Visualization in PyPSA

This tutorial covers how to visualize PyPSA networks using various plotting functions. We'll learn how to create static and interactive visualizations of network components, flows, and results.

## Prerequisites

Before starting this tutorial, you should have:
- Basic understanding of PyPSA components and network structure
- Familiarity with matplotlib and cartopy for plotting
- Understanding of geographic coordinates and projections
- Basic knowledge of pandas and numpy

## Basic Network Plot

Let's create a basic plot of the network using PyPSA's built-in plotting function. The `plot()` method provides several options for customizing the visualization:

In [1]:
import pypsa
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Create network
network = pypsa.Network()
network.set_snapshots(pd.date_range("2025-05-01", periods=24, freq="h"))

# Add buses with coordinates
for i in range(3):
    network.add(
        "Bus",
        f"bus {i}",
        v_nom=20.0,
        x=i,  # x-coordinate
        y=i   # y-coordinate
    )

# Add carriers with colors
network.add(
    "Carrier",
    ["hydro", "solar"],
    co2_emissions=[0.2, 0.0],
    nice_name=["Natural Gas", "Solar PV"],
    color=["indianred", "yellow"]
)

# Add generators
network.add(
    "Generator",
    "gas_plant",
    bus="bus 0",
    p_nom=100,
    carrier="hydro",
    marginal_cost=50,
    capital_cost=1000
)

network.add(
    "Generator",
    "solar",
    bus="bus 1",
    p_nom=50,
    carrier="solar",
    marginal_cost=0,
    capital_cost=800
)

# Add load with time-varying profile
load_profile = pd.Series(
    [0.7 + 0.3 * np.sin(hour * np.pi/12) for hour in range(24)],
    index=network.snapshots
)

network.add(
    "Load",
    "load",
    bus="bus 2",
    p_set=50 * load_profile
)

# Add lines with realistic parameters
for i in range(3):
    network.add(
        "Line",
        f"line_{i}",
        bus0=f"bus {i}",
        bus1=f"bus {(i + 1) % 3}",
        s_nom=100,
        x=0.1,  # reactance
        r=0.01,  # resistance
        g=0.0,   # conductance
        b=0.001  # susceptance
    )

## Basic Network Plot

Let's create a basic plot of the network:

In [ ]:
# Create a basic network plot
network.plot()
plt.title("Basic Network Plot")
plt.show()

## Customized Network Plot with Component Sizes

We can customize the plot to show component capacities and flows. Here's an example with more advanced options:

In [ ]:
# Create a customized network plot
network.plot(
    bus_sizes=0.01,  # Size of bus markers
    line_widths=2,   # Width of lines
    bus_colors='lightblue',  # Color of buses
    line_colors='gray',      # Color of lines
    title='Customized Network Plot'
)
plt.show()

## Plotting with Geographic Data

PyPSA can plot networks on geographic maps using the `cartopy` library. This requires bus coordinates (`x`, `y`) to represent longitude and latitude (usually in decimal degrees).

**Note on Cartopy Installation:** `cartopy` has external dependencies (like GEOS and PROJ) and might require more setup than typical Python packages. Please refer to the [Cartopy installation guide](https://scitools.org.uk/cartopy/docs/latest/installing.html) if you encounter issues.

Let's create a network with geographic coordinates for some cities in Kenya:

In [11]:
import cartopy.crs as ccrs
import cartopy.feature as cfeature

# Create a network with geographic coordinates
network_geo = pypsa.Network()
network_geo.set_snapshots(pd.date_range("2025-04-01", periods=24, freq="h"))

# Add buses with geographic coordinates (example: cities in Kenya)
buses = {
    "Busia": (34.1299, 0.4347),
    "Mombasa": (39.6682, -4.0435),
    "Mandera": (41.8670, 3.9373)
}

for city, (lon, lat) in buses.items():
    network_geo.add(
        "Bus",
        city,
        v_nom=220.0,
        x=lon,
        y=lat
    )

# Add carriers with colors
network_geo.add(
    "Carrier",
    ["gas", "solar", "wind"],
    co2_emissions=[0.0, 0.0, 0.0],
    nice_name=["Gas", "Solar PV", "Wind"],
    color=["indianred", "yellow", "dodgerblue"]
)

# Add generators
network_geo.add(
    "Generator",
    "gas_plant",
    bus="Busia",
    p_nom=200,
    p_nom_extendable=True,
    carrier="gas",
    marginal_cost=50,
    capital_cost=1000
)

network_geo.add(
    "Generator",
    "solar",
    bus="Mombasa",
    p_nom=100,
    p_nom_extendable=True,
    carrier="solar",
    marginal_cost=0,
    capital_cost=800
)

# Add load with time-varying profile
load_profile = pd.Series(
    [0.3, 0.2, 0.1, 0.1, 0.2, 0.5, 0.7, 1.0, 1.2, 1.0, 0.9, 0.8,
     0.7, 0.6, 0.6, 0.7, 0.9, 1.1, 1.3, 1.0, 0.8, 0.6, 0.4, 0.3],
    index=network_geo.snapshots
)

network_geo.add(
    "Load",
    "load",
    bus="Mandera",
    p_set=40 * load_profile
)

# Add lines between cities (parameters are illustrative)
cities = list(buses.keys())
for i in range(len(cities)):
    network_geo.add(
        "Line",
        f"line_{i}",
        bus0=cities[i],
        bus1=cities[(i + 1) % len(cities)],
        s_nom=200, # Illustrative capacity (MVA)
        s_nom_extendable=True,
        # Note: x, r, b are given here as TOTAL per-unit values for the line segment.
        # Alternatively, specify per-km values (e.g., type='...') referencing a standard line type
        # or manually providing x_per_km, r_per_km, b_per_km, along with 'length'.
        x=0.2,  # Illustrative TOTAL reactance (p.u.) based on system V and S base
        r=0.05, # Illustrative TOTAL resistance (p.u.)
        g=0.0,  # Conductance (often negligible)
        b=0.001,# Illustrative TOTAL susceptance (p.u.)
        capital_cost=1000, # Illustrative cost (e.g., per MVA or per distance unit)
        # Length here is informational since x, r, b are total p.u. values.
        # If using per-km impedance, length is crucial for calculation.
        length=100 # Illustrative length (km)
    )

In [ ]:
# Create geographic plot with custom projection
fig, ax = plt.subplots(figsize=(12, 10), subplot_kw={'projection': ccrs.PlateCarree()})

# Plot the network on the map
network_geo.plot(
    ax=ax,
    title='Network on Geographic Map',
    color_geomap=True,
    bus_sizes=0.1,
    line_widths=2,
    boundaries=[33, 42, -5, 5],
    geomap='10m',
    projection=ccrs.PlateCarree(),
)

# Add map features
ax.gridlines(draw_labels=True)
ax.coastlines(resolution='10m')
ax.add_feature(cfeature.BORDERS, linestyle=':')
ax.add_feature(cfeature.LAND, alpha=0.1)
ax.add_feature(cfeature.OCEAN, alpha=0.1)

plt.tight_layout()
plt.show()

## Interactive Network Visualization

PyPSA supports interactive visualization using plotly. This allows for dynamic exploration of the network, including zooming, panning, and hovering over components to see their details. Note that you need to install nbformat>=4.2.0 and plotly for this to work:

```bash
# You might need to run this in your terminal or notebook environment
# pip install plotly nbformat>=4.2.0
```

In [ ]:
# Import necessary libraries for interactive visualization
import plotly.graph_objects as go
from plotly.offline import iplot

# Create an interactive plot using the built-in iplot method
network_geo.iplot(
    bus_sizes=0.2,  # Increased for better visibility
    line_widths=3,  # Increased for better visibility
    title='Interactive Network Plot',
    jitter=0,
    line_colors='grey',  # Add consistent line colors
    bus_colors='red',    # Add bus colors
)

## Plotting Network Statistics

We can also visualize network statistics and results using various plotting methods:

In [ ]:
# Run optimization (e.g., LOPF) to generate results for plotting
# This calculates optimal capacities (p_nom_opt, s_nom_opt) and dispatch (generators_t.p, lines_t.p0 etc.)
network_geo.optimize()


# Create a figure with multiple subplots
fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# Plot 1: Generator capacities (use p_nom_opt for optimized capacities)
network_geo.generators.p_nom_opt.plot(
    kind='bar',
    ax=axes[0, 0],
    title='Optimized Generator Capacities',
    ylabel='Capacity (MW)',
    color=network_geo.generators.carrier.map(network_geo.carriers.color) # Use network_geo carriers
)
axes[0, 0].tick_params(axis='x', rotation=45)

# Plot 2: Line capacities (use s_nom_opt for optimized capacities)
network_geo.lines.s_nom_opt.plot(
    kind='bar',
    ax=axes[0, 1],
    title='Optimized Line Capacities',
    ylabel='Capacity (MVA)'
)
axes[0, 1].tick_params(axis='x', rotation=45)

# Plot 3: Load profile
network_geo.loads_t.p_set.plot(
    ax=axes[1, 0],
    title='Load Profile',
    ylabel='Power (MW)',
    legend=True
)

# Plot 4: Generator dispatch (results from optimization)
network_geo.generators_t.p.plot(
    ax=axes[1, 1],
    title='Generator Dispatch',
    ylabel='Power (MW)',
    legend=True
)

plt.tight_layout()
plt.show()

## Advanced Plotting Features

### 1. Flow Visualization

You can visualize power flows on the network using the `flow` parameter. We'll plot the mean flows on our geographic network (`network_geo`) which we optimized earlier.

In [ ]:
# Set default figure size for this section
plt.rc("figure", figsize=(12, 10)) # Adjusted size for geographic plot

# Calculate mean line flows from the optimization results of network_geo
# We need to create a MultiIndex Series for the flows
mean_flows = network_geo.lines_t.p0.mean()
mean_flows.index = pd.MultiIndex.from_product([['Line'], mean_flows.index])

# Create geographic plot axes
proj = ccrs.PlateCarree()
fig, ax = plt.subplots(subplot_kw={"projection": proj})

# Plot network_geo with mean flows
network_geo.plot(
    ax=ax,
    bus_sizes=0.01, # Adjust size as needed
    flow=mean_flows, # Pass the MultiIndexed mean flows
    line_widths=3, #network_geo.lines.s_nom_opt / 100, # Example: Width proportional to optimized capacity
    link_widths=0, # Assuming no links in this network
    bus_colors='blue', # Simple color
    line_colors='grey', # Base line color
    color_geomap=True, # Show geographic background
    boundaries=[33, 42, -5, 5], # Kenya boundaries
    geomap='10m',
    projection=proj,
    title='Geographic Network with Mean Power Flows'
)

# Add map features
ax.gridlines(draw_labels=True)
ax.coastlines(resolution='10m')
ax.add_feature(cfeature.BORDERS, linestyle=':')
ax.add_feature(cfeature.LAND, alpha=0.1)
ax.add_feature(cfeature.OCEAN, alpha=0.1)

plt.tight_layout()
plt.show()

### 2. Custom Layouts

You can use NetworkX layout algorithms to automatically position network components. This is useful for understanding the network's topology abstractly, without geographic constraints, especially for complex or non-geographic networks.

In [ ]:
import networkx as nx
import cartopy.crs as ccrs

# Create a plot with custom layout using geographic projection
fig, ax = plt.subplots(figsize=(10, 8), subplot_kw={"projection": ccrs.PlateCarree()})

# Plot network using spring layout
network.plot(
    ax=ax,
    layouter=nx.spring_layout,
    title='Network with Spring Layout',
    bus_sizes=0.01,
    line_widths=2
)

ax.legend()

plt.tight_layout()
plt.show()

### 3. Split Circle Visualization

For buses with multiple components (like different types of generators), you can use split circles to show the composition, often representing relative capacities.

In [ ]:
# Create a plot with split circles using geographic projection
fig, ax = plt.subplots(figsize=(10, 8), subplot_kw={"projection": ccrs.PlateCarree()})

# Calculate bus composition
bus_composition = network.generators.groupby(['bus', 'carrier']).p_nom.sum()

# Plot network with split circles
network.plot(
    ax=ax,
    bus_sizes=bus_composition,
    bus_split_circles=True,
    title='Network with Split Circles',
    line_widths=1
)

ax.legend()

plt.tight_layout()
plt.show()

## Plotting Options Reference

Key plotting options in PyPSA:

### Basic Options
- `bus_sizes`: Size of bus markers (float or array)
- `line_widths`: Width of transmission lines (float or array)
- `bus_colors`: Color of buses (str, array, or dict)
- `line_colors`: Color of lines (str, array, or dict)
- `title`: Plot title (str)
- `legend`: Show legend (bool)
- `legend_kwargs`: Legend parameters (dict)
- `margin`: Margin around the plot (float)
- `ax`: Matplotlib axis object (optional)
- `show`: Whether to show the plot (bool)

### Geographic Options
- `color_geomap`: Show geographic background (bool)
- `geomap`: Specify map style ('10m', '50m', '110m')
- `boundaries`: Set map boundaries [lon_min, lon_max, lat_min, lat_max]
- `projection`: Map projection (cartopy.crs.Projection)
- `coastlines`: Show coastlines (bool)
- `borders`: Show country borders (bool)
- `land`: Show land with transparency (bool)
- `ocean`: Show ocean with transparency (bool)

### Component Options
- `show_generators`: Show generator locations (bool)
- `show_loads`: Show load locations (bool)
- `show_storage`: Show storage locations (bool)
- `show_links`: Show link locations (bool)
- `show_lines`: Show line locations (bool)
- `show_transformers`: Show transformer locations (bool)
- `show_shunt_impedances`: Show shunt impedance locations (bool)

### Interactive Options
- `iplot()`: Create interactive plotly visualization
- `explore()`: Create interactive network explorer

### Flow Visualization Options
- `flow`: Power flow data to visualize (pd.Series)
- `flow_max`: Maximum flow value for scaling (float)
- `flow_min`: Minimum flow value for scaling (float)
- `flow_colors`: Color scheme for flows (str or list)
- `flow_scale`: Scale factor for flow arrows (float)

### Additional Features
- `add_legend_circles()`: Add legend circles for bus sizes
- `add_legend_patches()`: Add legend patches for component types
- `add_legend_lines()`: Add legend lines for flow scales
- `add_colorbar()`: Add colorbar for flow visualization
- `add_scalebar()`: Add scale bar for geographic plots

## Best Practices

1. **Figure Size and Layout**
   - Use appropriate figure sizes for your visualization needs
   - Apply `plt.tight_layout()` to prevent overlapping elements
   - Consider using subplots for complex visualizations

2. **Color Schemes**
   - Use consistent colors for similar components
   - Consider colorblind-friendly palettes
   - Use alpha transparency for overlapping elements

3. **Geographic Plots**
   - Choose appropriate map projections for your region
   - Add relevant geographic features (coastlines, borders)
   - Use appropriate map resolution for your needs

4. **Interactive Plots**
   - Add meaningful hover information
   - Use appropriate zoom levels
   - Consider adding interactive controls

5. **Performance**
   - For large networks, consider using lower resolution maps
   - Use appropriate data aggregation for time series
   - Consider using static plots for large datasets

## Next Steps

In the next tutorial, we'll learn how to:
- Perform network optimization
- Analyze optimization results
- Implement contingency analysis
- Create custom visualizations for specific analysis needs